In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import preprocessor as p
from collections import defaultdict
from nltk.corpus import stopwords
import tweepy
from nltk.stem import PorterStemmer, LancasterStemmer
import codecs, json, string, re, time, os 

    #RAW DATA: 

In [3]:
#MERGE USERS: 
d1=pd.read_csv('~/analysis/dyads/data/unverified_dyads.csv')
d0=pd.read_csv('~/analysis/dyads/data/random_one_edges.csv')
d1.loc[:,'Class'] = np.ones(len(d1), dtype=np.int16) 
d0.loc[:,'Class'] = np.zeros(len(d0), dtype=np.int16) 
df2 = d1.append(d0)
df2.head(1)

,Dyad,B1_ID,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,...,B2_Friends,B2_Statuses,B2_Description,B2_Location,B2_Favourite,B2_Verification,B2_Tweet,B2_AccountCreated,B2_RTCount,Class
0,"('ArmaniStation', 'mxslimxh')",966710486.0,ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,...,80.0,292.0,أهل السنة والجماعة,NaN,12876.0,False,@YaqubAlRashid1 Yea I also read that Hadith! T...,2019-03-06 20:54:02,0.0,1


In [4]:
pairs = []
for a , b in zip(df2.B1_ScreenName, df2.B2_ScreenName):
    a = str(a)
    b = str(b)
    
    a = a[:3]+str(len(a))
    b = b[:3]+str(len(b))
    pairs.append((a,b))
df2['Pairs'] = pairs

In [6]:
df1 = df2[['Pairs','B1_ScreenName','B1_CreatedAt', 'B1_Followers','B1_Friends', 'B1_Statuses',
 'B1_Description','B1_Location','B1_Favourite','B1_Verification', 'B1_Tweet','B1_RTCount',
'B2_ScreenName','B2_CreatedAt', 'B2_Followers', 'B2_Friends', 'B2_Statuses', 'B2_Description',
'B2_Location','B2_Favourite', 'B2_Verification', 'B2_Tweet', 'B2_RTCount', 'Class']]
df1.head(1)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_Followers,B2_Friends,B2_Statuses,B2_Description,B2_Location,B2_Favourite,B2_Verification,B2_Tweet,B2_RTCount,Class
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,4039.0,80.0,292.0,أهل السنة والجماعة,NaN,12876.0,False,@YaqubAlRashid1 Yea I also read that Hadith! T...,0.0,1


    # ..... 

In [8]:
#Individual Reciprocity ... 
df1.loc[:,'B1_Recip'] = df1.B1_Friends/(df1.B1_Followers+df1.B1_Friends)
df1.loc[:,'B2_Recip'] = df1.B2_Friends/(df1.B2_Followers+df1.B2_Friends)
# 
df1.loc[:,'B1B2_Recip'] = (1/2)*(df1.B1_Recip*df1.B2_Recip)+df1.B1_Recip
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_Description,B2_Location,B2_Favourite,B2_Verification,B2_Tweet,B2_RTCount,Class,B1_Recip,B2_Recip,B1B2_Recip
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,أهل السنة والجماعة,NaN,12876.0,False,@YaqubAlRashid1 Yea I also read that Hadith! T...,0.0,1,0.104378,0.019422,0.105391
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,revert 🌱🕋,"South london, UK",9784.0,False,NaN,NaN,1,0.104378,0.132826,0.111310


In [11]:
pairMention = []

for name1, name2, tweet1, tweet2 in zip(df1.B1_ScreenName, df1.B2_ScreenName,\
                                        df1.B1_Tweet, df1.B2_Tweet):
    mention1 = [tok1 for tok1 in str(tweet1).split() if tok1.startswith('@')]
    mention2 = [tok2 for tok2 in str(tweet2).split() if tok2.startswith('@')]
    #m.append((mention1, mention2))
        
    if (name1 in mention2) and (name2 in mention1):
        pairMention.append(1)
    elif (name1 in mention2) or (name2 in mention1):
        pairMention.append(0.5)
    else:
        pairMention.append(0)
#A MENTION B:
df1.loc[:,'B1MentionB2'] = pairMention
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_Location,B2_Favourite,B2_Verification,B2_Tweet,B2_RTCount,Class,B1_Recip,B2_Recip,B1B2_Recip,B1MentionB2
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,NaN,12876.0,False,@YaqubAlRashid1 Yea I also read that Hadith! T...,0.0,1,0.104378,0.019422,0.105391,0
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,"South london, UK",9784.0,False,NaN,NaN,1,0.104378,0.132826,0.111310,0


In [12]:
pairRT = []
m1 = []
m2 = []
for name1, name2, tweet1, tweet2 in zip(df1.B1_ScreenName, df1.B2_ScreenName,\
                                        df1.B1_Tweet, df1.B2_Tweet):
    name1 = '@'+str(name1)
    name2 = '@'+str(name2)
    
    if (str(tweet1).split()[0] == 'RT') and str(tweet2).split()[0] == 'RT':
        #m1.append((name1, str(tweet2).split()[1][:-1]))
        #m2.append((name2, str(tweet1).split()[1][:-1]))
        if (name1 == str(tweet2).split()[1][:-1]) and (name2 == str(tweet1).split()[1][:-1]):
            pairRT.append(1)
            
            
        elif (name1 == str(tweet2).split()[1][:-1]) or (name2 == str(tweet1).split()[1][:-1]):       
            pairRT.append(0.5)
            
        else:
            pairRT.append(0)         
    else:
        pairRT.append(0)
            
#A RT B:
df1.loc[:,'B1RetweetB2'] = pairRT
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_Favourite,B2_Verification,B2_Tweet,B2_RTCount,Class,B1_Recip,B2_Recip,B1B2_Recip,B1MentionB2,B1RetweetB2
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,12876.0,False,@YaqubAlRashid1 Yea I also read that Hadith! T...,0.0,1,0.104378,0.019422,0.105391,0,0.0
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,9784.0,False,NaN,NaN,1,0.104378,0.132826,0.111310,0,0.0


     #Textual ... sentiment and text similarity ... 

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gensim
from gensim.summarization.summarizer import summarize
import sklearn
from sklearn.metrics.pairwise import cosine_similarity 

In [14]:
#SENTIMENT .... A_SENTIMENTTOWARDS_B:
# sentiment analyser:
analyser = SentimentIntensityAnalyzer()
# a basic function to extract extract the sentiment in each tweet: 
def tweet_sentiments(tweet):
    tweet = str(tweet)
    if tweet != None:
        score = analyser.polarity_scores(tweet)
        return score
# compute overall all sentiments and store:
%time df1.loc[:, 'B1_Sentiments']= df1.B1_Tweet.apply(tweet_sentiments)
%time df1.loc[:, 'B2_Sentiments']= df1.B2_Tweet.apply(tweet_sentiments)

CPU times: user 3.51 s, sys: 16.1 ms, total: 3.53 s
Wall time: 3.65 s
CPU times: user 3.47 s, sys: 35.7 ms, total: 3.51 s
Wall time: 3.51 s


In [ ]:
# include individual sentiments as features/columns
#B1: 
df1.loc[:,'B1_NegSent'] = df1.B1_Sentiments.apply(lambda x: x['neg'])
df1.loc[:,'B1_PosSent'] = df1.B1_Sentiments.apply(lambda x: x['pos'])
df1.loc[:,'B1_NeuSent'] = df1.B1_Sentiments.apply(lambda x: x['neu'])
df1.loc[:,'B1_CompoundSent'] = df1.B1_Sentiments.apply(lambda x: x['compound'])
#B2: 
df1.loc[:,'B2_NegSent'] = df1.B2_Sentiments.apply(lambda x: x['neg'])
df1.loc[:,'B2_PosSent'] = df1.B2_Sentiments.apply(lambda x: x['pos'])
df1.loc[:,'B2_NeuSent'] = df1.B2_Sentiments.apply(lambda x: x['neu'])
df1.loc[:,'B2_CompoundSent'] = df1.B2_Sentiments.apply(lambda x: x['compound'])

In [ ]:
#save:
# extract negative, neutral and postivie tweets:
dfpos1 = df1[df1['B1_CompoundSent']>0.05]
dfneg1 = df1[df1['B1_CompoundSent']<=-0.05]
dfneu1 = df1[(df1['B1_CompoundSent']> -0.05) & (df1['B1_CompoundSent']<0.05)]
#INCLUDE CLASSES: 
dfneg1.loc[:,'B1_SentClass']=[-1 for i in range(len(dfneg1))]
dfpos1.loc[:,'B1_SentClass']=np.ones(len(dfpos1), dtype='int8')
dfneu1.loc[:,'B1_SentClass']=np.zeros(len(dfneu1), dtype='int8')
dfsent1 = dfpos1.append([dfneg1,dfneu1])
#len(dfsent1)

In [17]:
#dfsent1.B1_SentClass.value_counts()

In [ ]:
#B2: 
dfpos2 = df1[df1['B2_CompoundSent']>0.05]
dfneg2 = df1[df1['B2_CompoundSent']<=-0.05]
dfneu2 = df1[(df1['B2_CompoundSent']> -0.05) & (df1['B2_CompoundSent']<0.05)]
#### 
dfneg2.loc[:,'B2_SentClass']=[-1 for i in range(len(dfneg2))]
dfpos2.loc[:,'B2_SentClass']=np.ones(len(dfpos2), dtype='int8')
dfneu2.loc[:,'B2_SentClass']=np.zeros(len(dfneu2), dtype='int8')
dfsent2 = dfpos2.append([dfneg2,dfneu2])
#len(dfsent2)

In [19]:
b1b2Sentiment = []
for sent1, sent2 in zip(dfsent1.B1_SentClass, dfsent2.B2_SentClass):
    
    #positive group/positive support or shared sentiment
    if (sent1==1) & (sent2==1):
        b1b2Sentiment.append(1)
    
    elif (sent1==0) & (sent2==0):
        b1b2Sentiment.append(1)
    
    elif (sent1==-1) & (sent2==-1):
        b1b2Sentiment.append(1)
    
    #oneway sentiment 
    elif (sent1==1) & (sent2==-1):
        b1b2Sentiment.append(0.5)
    
    elif (sent1==-1) & (sent2==1):
        b1b2Sentiment.append(0.5)
    
    elif (sent1==1) & (sent2==0):
        b1b2Sentiment.append(0.5)
    
    elif (sent1==0) & (sent2==-1):
        b1b2Sentiment.append(0.5)
    #no sentiment ... 
    else:
        b1b2Sentiment.append(0)
df1.loc[:,'B1B2SentClass'] = b1b2Sentiment
df1.head(1)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_Sentiments,B1_NegSent,B1_PosSent,B1_NeuSent,B1_CompoundSent,B2_NegSent,B2_PosSent,B2_NeuSent,B2_CompoundSent,B1B2SentClass
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


    #SHARED TEXT ....... 

In [20]:
from collections import Counter
import json, string,codecs
#LARGE_FIGSIZE = (16,11)
import re
import gensim
from gensim.models.word2vec import Word2Vec
import preprocessor as p
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
b1tweets = df1.B1_Tweet.tolist()
b2tweets = df1.B2_Tweet.tolist()
#len(b1tweets), len(b2tweets)

In [22]:
#two different models for both b1 and b2 texts ... 
model1 = Word2Vec()
model1.build_vocab([str(tweet).split() for tweet in b1tweets])
#B2: 
model2 = Word2Vec()
model2.build_vocab([str(tweet).split() for tweet in b2tweets])

In [23]:
%time model1.train([str(tweet).split() for tweet in b1tweets], total_examples=len(b1tweets), epochs=21)
%time model2.train([str(tweet).split() for tweet in b2tweets], total_examples=len(b2tweets), epochs=21)

CPU times: user 13.2 s, sys: 79.7 ms, total: 13.3 s
Wall time: 5.13 s
CPU times: user 7.25 s, sys: 27.7 ms, total: 7.28 s
Wall time: 3 s


(2112239, 4448850)

In [24]:
#model1.wv.most_similar('good')

Build a tfdif model to weigh the relavance of words in the corpus to be used in approaximating tweets

In [25]:
# use averaging based on terms weigthing based on tfidf: OR USING raw averaging of vectors
def mean_vectors1(tweets, size):
    vectors = np.zeros(size).reshape((1,size))
    count = 0
    for tweet in tweets:
        tweet = p.clean(tweet)
        for token in tweet.split():
            try:
                vectors +=model1[token].reshape((1,size))
                count +=1
            except KeyError:
                continue
    if count !=0:
        vectors/=count
    return vectors

#using model2: 
def mean_vectors2(tweets, size):
    vectors = np.zeros(size).reshape((1,size))
    count = 0
    for tweet in tweets:
        tweet = p.clean(tweet)
        for token in tweet.split():
            try:
                vectors +=model2[token].reshape((1,size))
                count +=1
            except KeyError:
                continue
    if count !=0:
        vectors/=count
    return vectors

cancatenate word vectors to reproduce the equivalent tweet ... # using the raw values of the vectors to represent tweet vector 

In [ ]:
# using the raw values of the vectors to represent tweet vector ... use: vectors
%time b1tweetsw2v1=np.concatenate([mean_vectors1(k,100) for k in map(lambda x: str(x), b1tweets)])
%time b2tweetsw2v1=np.concatenate([mean_vectors2(k,100) for k in map(lambda x: str(x), b2tweets)])

In [27]:
#B1:
df1['B1_TweetsW2V'] = [np.array(np.round(x,2), dtype=np.float) for x in b1tweetsw2v1]
#B2: 
df1['B2_TweetsW2V'] = [np.array(np.round(x,2), dtype=np.float) for x in b2tweetsw2v1]
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B1_PosSent,B1_NeuSent,B1_CompoundSent,B2_NegSent,B2_PosSent,B2_NeuSent,B2_CompoundSent,B1B2SentClass,B1_TweetsW2V,B2_TweetsW2V
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[-0.06, 0.12, 0.21, -0.03, -0.09, -0.06, 0.07,..."
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[0.03, 0.12, 0.09, -0.07, 0.0, -0.01, 0.25, 0...."


In [28]:
# COMPUTE THE PAIRWISE COSIM SIMILARITY: 
b1b2TextSim = []
for b1vector, b2vector in zip(df1.B1_TweetsW2V, df1.B2_TweetsW2V):
    b1b2TextSim.append(np.round(cosine_similarity(b1vector.reshape(1,-1),\
                                                  b2vector.reshape(1,-1)),2)[0][0])

df1['B1B2TextSim'] = b1b2TextSim
df1['B1B2TextSim1'] = df1.B1B2TextSim.apply(lambda x: 1 if x > 0.65 else 0)
df1.head(1)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B1_CompoundSent,B2_NegSent,B2_PosSent,B2_NeuSent,B2_CompoundSent,B1B2SentClass,B1_TweetsW2V,B2_TweetsW2V,B1B2TextSim,B1B2TextSim1
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,0.0,0.0,0.0,1.0,0.0,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[-0.06, 0.12, 0.21, -0.03, -0.09, -0.06, 0.07,...",0.67,1


In [ ]:
df1['B1_GlobalInterest'] = df1.B1_Favourite/df1.B1_Statuses
df1['B2_GlobalInterest'] = df1.B2_Favourite/df1.B2_Statuses

    #Common opinion .... 

In [29]:
b1b2CommonOpinion = []
for b1b2sim, b1b2sent in zip(df1.B1B2TextSim, df1.B1B2SentClass):
    if b1b2sim == b1b2sent:
        b1b2CommonOpinion.append(1)
    elif (b1b2sim == 1) & (b1b2sent == 0.5):
        b1b2CommonOpinion.append(1)
    else:
        b1b2CommonOpinion.append(0)
# ... 
df1['B1B2ComOpinion'] = b1b2CommonOpinion
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_NegSent,B2_PosSent,B2_NeuSent,B2_CompoundSent,B1B2SentClass,B1_TweetsW2V,B2_TweetsW2V,B1B2TextSim,B1B2TextSim1,B1B2ComOpinion
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,0.0,0.0,1.0,0.0,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[-0.06, 0.12, 0.21, -0.03, -0.09, -0.06, 0.07,...",0.67,1,0
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,0.0,0.0,1.0,0.0,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[0.03, 0.12, 0.09, -0.07, 0.0, -0.01, 0.25, 0....",0.52,0,0


In [34]:
#HASHTAGS: 
df1.loc[:,'B1_Hashtags'] = df1.B1_Tweet.apply(lambda x: get_hashtags(x))
df1.loc[:,'B2_Hashtags'] = df1.B2_Tweet.apply(lambda x: get_hashtags(x))
#USER MENTION: 
df1.loc[:,'B1_Mentions'] = df1.B1_Tweet.apply(lambda x: get_usermention(x))
df1.loc[:,'B2_Mentions'] = df1.B2_Tweet.apply(lambda x: get_usermention(x))
df1.head(2)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B1B2SentClass,B1_TweetsW2V,B2_TweetsW2V,B1B2TextSim,B1B2TextSim1,B1B2ComOpinion,B1_Hashtags,B2_Hashtags,B1_Mentions,B2_Mentions
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[-0.06, 0.12, 0.21, -0.03, -0.09, -0.06, 0.07,...",0.67,1,0,[],[],[],[@yaqubalrashid1]
1,"(Arm13, diy7)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,1.0,"[-0.01, -0.11, 0.56, -0.04, -0.5, 0.27, 0.31, ...","[0.03, 0.12, 0.09, -0.07, 0.0, -0.01, 0.25, 0....",0.52,0,0,[],[],[],[]


In [35]:
#EXACT MATCHING: Hashtag between pairs: 
similar_htags = []
shared_htags = []
for h1, h2 in zip(df1.B1_Hashtags, df1.B2_Hashtags):
    #check for the smallest list:
    
    sh_h1h2 = []
    sim_h1h2 = []
    if (len(h1)==0) or (len(h2)==0):
        sh_h1h2.append(np.nan)
        sim_h1h2.append('No')            
        
    elif (len(h1)>0) or (len(h2)>0):
        for h in h1: # check for common hashtags between h1 & h2 ... 
            if h in h2:
                sh_h1h2.append((h1,h2))
                sim_h1h2.append('Yes')            
                
    else:
        sh_h1h2.append('None')
        sim_h1h2.append('No')
        
        
    shared_htags.append((sh_h1h2)) # or (h,h2)
    similar_htags.append(sim_h1h2) # or (h,h2)
    
#more column features: 
df1['SharedHtags'] = shared_htags
df1['SimilarHtags'] = similar_htags
df1.head(1)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B2_TweetsW2V,B1B2TextSim,B1B2TextSim1,B1B2ComOpinion,B1_Hashtags,B2_Hashtags,B1_Mentions,B2_Mentions,SharedHtags,SimilarHtags
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,"[-0.06, 0.12, 0.21, -0.03, -0.09, -0.06, 0.07,...",0.67,1,0,[],[],[],[@yaqubalrashid1],[nan],[No]


In [36]:
#EXACT MATCHING: Hashtag between pairs ... user mentions: 
similar_mention = []
shared_mention = []
for m1, m2 in zip(df1.B1_Mentions, df1.B2_Mentions):
    #check for the smallest list:
    
    sh_m1m2 = []
    sim_m1m2 = []
    if (len(m1)==0) or (len(m2)==0):
        sh_m1m2.append(np.nan)
        sim_m1m2.append('No')            
        
    elif (len(m1)>0) or (len(m2)>0):
        for m in m1: # check for common hashtags between h1 & h2 ... 
            if m in m2:
                sh_m1m2.append((m1,m2))
                sim_m1m2.append('Yes')            
                
    else:
        sh_m1m2.append('None')
        sim_m1m2.append('No')
        
        
    shared_mention.append((sh_m1m2)) # or (h,h2)
    similar_mention.append(sim_m1m2) # or (h,h2)

#more column features: 
df1['SharedMentions'] = shared_mention
df1['SimilarMentions'] = similar_mention
df1.head(1)

,Pairs,B1_ScreenName,B1_CreatedAt,B1_Followers,B1_Friends,B1_Statuses,B1_Description,B1_Location,B1_Favourite,B1_Verification,...,B1B2TextSim1,B1B2ComOpinion,B1_Hashtags,B2_Hashtags,B1_Mentions,B2_Mentions,SharedHtags,SimilarHtags,SharedMentions,SimilarMentions
0,"(Arm13, mxs8)",ArmaniStation,2012-11-23 19:38:18,4153.0,484.0,14942.0,No bio,The Netherlands/London,5338.0,False,...,1,0,[],[],[],[@yaqubalrashid1],[nan],[No],[nan],[No]


In [38]:
#SAVE FILE:df1.to_csv('data/fullset-of-reciprocated-data-class10.csv', index_label=False)

    #TRAINING DATA .... 

In [41]:
#training data1: dt.to_csv('data/ml-training-dataset.csv', index_label=False)
dt = dt05.append([dt0[:600], dt1[:1100]])
len(dt)

2107

In [42]:
#THE TRAINING DATA: 
dt = df1[['Pairs','B1_Recip', 'B2_Recip', 'B1B2_Recip','B1MentionB2', 'B1RetweetB2',
          'B1_CompoundSent','B2_CompoundSent','B1B2TextSim','Class']]
dt.head(1)

,Pairs,B1_Recip,B2_Recip,B1B2_Recip,B1MentionB2,B1RetweetB2,B1_CompoundSent,B2_CompoundSent,B1B2TextSim,Class
0,"(Arm13, mxs8)",0.104378,0.019422,0.105391,0,0.0,0.0,0.0,0.67,1


In [43]:
dt

,Pairs,B1_Recip,B2_Recip,B1B2_Recip,B1MentionB2,B1RetweetB2,B1_CompoundSent,B2_CompoundSent,B1B2TextSim,Class
0,"(Arm13, mxs8)",0.104378,0.019422,0.105391,0,0.0,0.00,0.0000,0.67,1
1,"(Arm13, diy7)",0.104378,0.132826,0.111310,0,0.0,0.00,0.0000,0.52,1
2,"(Arm13, TWl14)",0.104378,0.075410,0.108313,0,0.0,0.00,0.2960,0.62,1
3,"(Arm13, rak9)",0.104378,0.460711,0.128422,0,0.0,0.00,0.0000,0.69,1
0,"(shp11, hha9)",0.165293,0.725490,0.225252,0,0.0,0.00,0.4404,0.55,1
...,...,...,...,...,...,...,...,...,...,...
2,"(Bro14, Red13)",0.521357,0.639430,0.688043,0,0.0,-0.34,0.3818,0.69,0
3,"(Bro14, Civ13)",0.521357,0.497455,0.651033,0,0.0,-0.34,0.0000,0.69,0
4,"(Bro14, big11)",0.521357,0.508095,0.653806,0,0.0,-0.34,0.4995,0.67,0
5,"(Bro14, fra11)",0.521357,0.485270,0.647856,0,0.0,-0.34,0.0000,0.68,0


In [33]:
#SOME HELPER FUNCTIONS: 
def get_usermention(x):
    x = str(x)
    return [str(i).lower() for i in x.split() if i.startswith('@')]

def get_hashtags(x):
    x = str(x)
    return [str(i).lower() for i in x.split() if i.startswith('#')]

# extract the strings associalted with each hashtag, which will be later augmented with the main text (AugmentedS)
def get_hashtagstring(hashtags):
    return ' '.join([h[1:] for h in hashtags])

In [44]:
##########################################################################################